In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

# Occasions Annotations

## Bin of 3-5 `guest_count`
Let's look at a few tables:
- `korea_toronto_hocky_20190410_20190424.csv`
- `toronto_silvester_20181223_20190105.csv`
- `toronto_valentine_20190210_20190216.csv`

to get an idea of the different occasions.

Our initial set of **occasions** for the 3-5 bin is:
1. `Business Date`
2. `Dinner Date`
3. `Social Gathering`
4. `Family Event`
5. `Party`
6. `Drinking`
7. `After Work`

Something to think about:
- `Lunch`
- `Dinner`

----------

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
hocky_path = '../data/korea_toronto_hocky_20190410_20190424.csv'
silvester_path = '../data/toronto_silvester_20181223_20190105.csv'
valentine_path = '../data/toronto_valentine_20190210_20190216.csv'

In [5]:
hocky = pd.read_csv(hocky_path)
silvester = pd.read_csv(silvester_path)
valentine = pd.read_csv(valentine_path)

Fix `datetime` features:

In [6]:
def fix_times(df):
    df['order_time'] = pd.to_datetime(df['order_time'], format="%Y-%m-%d %H:%M:%S.%f")
    df['order_time_closed'] = pd.to_datetime(df['order_time_closed'], format="%Y-%m-%d %H:%M:%S.%f")
    df['order_item_time'] = pd.to_datetime(df['order_item_time'], format="%Y-%m-%d %H:%M:%S.%f")

In [7]:
fix_times(hocky)
fix_times(valentine)
fix_times(silvester)

Create `dwell_time` feature:

In [8]:
def set_dwell_time(df):
    df['dwell_time'] = df['order_time_closed'] - df['order_time']

In [9]:
set_dwell_time(hocky)
set_dwell_time(valentine)
set_dwell_time(silvester)

Drop the `bar_id` feature, it's useless here.

In [10]:
features = ['order_id', 'order_time', 'order_time_closed', 'dwell_time', 'order_item_id', 'order_item_time',
            'title', 'category_id', 'beer_brand_id', 'beer_serving_type_id', 'beer_volume',
            'item_qty', 'sales_before_tax', 'sales_inc_tax', 'guest_count']

Reorder columns:

In [11]:
hocky = hocky[features]
valentine = valentine[features]
silvester = silvester[features]

In [12]:
hocky.head()

,order_id,order_time,order_time_closed,dwell_time,order_item_id,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
0,520008106,2019-04-16 12:01:00,2019-04-16 13:15:52,01:14:52,52000810601,2019-04-16 12:13:26,$Sub.Swt.Pot.Fry-,2.0,0,0,0.0,1.0,2.97,3.425081,1
1,520008106,2019-04-16 12:01:00,2019-04-16 13:15:52,01:14:52,52000810600,2019-04-16 12:13:26,FINGERS.SAUCED,2.0,0,0,0.0,1.0,16.87,19.454919,1
2,520008105,2019-04-16 12:01:00,2019-04-16 13:11:55,01:10:55,52000810502,2019-04-16 12:18:25,TRAD.GUAC,2.0,0,0,0.0,1.0,8.99,10.368399,1
3,520008105,2019-04-16 12:01:00,2019-04-16 13:11:55,01:10:55,52000810501,2019-04-16 12:13:26,Add.Veg--,2.0,0,0,0.0,1.0,0.00,0.000000,1
4,520008105,2019-04-16 12:01:00,2019-04-16 13:11:55,01:10:55,52000810500,2019-04-16 12:13:26,VEGGIE.BURGER,2.0,0,0,0.0,1.0,17.62,20.321601,1


Create table for each order, we're going to pick (randomly) 10 tables out of each dataframe.

In [13]:
def create_tables(df):
    tables = df.drop_duplicates(subset='order_id', keep='first')
    tables.set_index('order_id', inplace=True)
    tables = tables[['guest_count']]

    return tables

In [14]:
hocky_tables = create_tables(hocky)
valentine_tables = create_tables(valentine)
silvester_tables = create_tables(silvester)

Keep only relevant features:

In [15]:
hocky.drop(['order_item_id'], axis=1, inplace=True)
valentine.drop(['order_item_id'], axis=1, inplace=True)
silvester.drop(['order_item_id'], axis=1, inplace=True)

In [16]:
hocky.set_index('order_id', inplace=True)
valentine.set_index('order_id', inplace=True)
silvester.set_index('order_id', inplace=True)

-----

## The Annotations

Pick 10 tables out of each dataframe.

In [17]:
COUNT = 10

In [18]:
import random

In [19]:
def get_tables_3to5(tables):
    return tables[(tables['guest_count'] >= 3) & (tables['guest_count'] <= 5)]

### Hocky

Pick 10 tables of 3 to 5 guests of **Hocky**.

In [20]:
hocky_tables_3to5 = get_tables_3to5(hocky_tables)
print("We have {} tables of 3 to 5 guests".format(len(hocky_tables_3to5)))

We have 13640 tables of 3 to 5 guests


In [21]:
# picked_hocky_tables = random.sample(list(hocky_tables_3to5.index), COUNT)

In [22]:
picked_hocky_tables = [
    520803608,
    523294848,
    514583822,
    524421347,
    521095118,
    522574215,
    522779155,
    515363895,
    519501788,
    514529424
]

In [23]:
print("Got {} tables".format(len(picked_hocky_tables)))

Got 10 tables


#### 1. Table 520803608

In [24]:
hocky[hocky.index == 520803608].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
520803608,2019-04-17 13:28:24,2019-04-17 14:44:51,01:16:27,2019-04-17 13:29:13,Steak & Eggs,2.0,0,0,0.000,1.0,15.99,18.069068,3
520803608,2019-04-17 13:28:24,2019-04-17 14:44:51,01:16:27,2019-04-17 13:29:13,Luau Chicken,2.0,0,0,0.000,1.0,15.99,18.069068,3
520803608,2019-04-17 13:28:24,2019-04-17 14:44:51,01:16:27,2019-04-17 13:29:13,Freddy Tuna Sal,2.0,0,0,0.000,1.0,12.99,14.678999,3
520803608,2019-04-17 13:28:24,2019-04-17 14:44:51,01:16:27,2019-04-17 13:29:13,Daily Soup,2.0,0,0,0.000,1.0,4.99,5.638815,3
520803608,2019-04-17 13:28:24,2019-04-17 14:44:51,01:16:27,2019-04-17 13:31:40,8oz Pinot Grigio,6.0,0,0,0.000,1.0,10.25,11.582736,3
520803608,2019-04-17 13:28:24,2019-04-17 14:44:51,01:16:27,2019-04-17 13:31:40,Von Bugle TB,1.0,8462,3,0.473,1.0,4.00,4.520092,3
520803608,2019-04-17 13:28:24,2019-04-17 14:44:51,01:16:27,2019-04-17 13:31:40,Von Bugle TB,1.0,8462,3,0.473,1.0,4.00,4.520092,3
520803608,2019-04-17 13:28:24,2019-04-17 14:44:51,01:16:27,2019-04-17 14:38:02,Coffee,4.0,0,0,0.000,1.0,2.69,3.039762,3
520803608,2019-04-17 13:28:24,2019-04-17 14:44:51,01:16:27,2019-04-17 14:38:02,Espresso DBL,4.0,0,0,0.000,1.0,2.75,3.107563,3


April 17th (Wednesday), 13:28 - 14:44, 3 guests.

Meal Steps Description:
1. Start
    - 3 Dishes (+1 Small Soup):
        - Steak & Eggs
        - Luau Chicken
        - Freddy Tuna Sal
        - Daily Soup
    - 3 Drinks:
        - Pinot Grigio Wine
        - Von Bugle Beer Can x 2
2. One Hour Later:
    - 3 Coffee Drinks:
        - Coffee
        - Espresso Double
        - Cafe Latte

`Lunch`

#### 2. Table 523294848

In [25]:
hocky[hocky.index == 523294848].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
523294848,2019-04-20 11:57:00,2019-04-20 12:40:43,00:43:43,2019-04-20 11:57:19,FAJ.CHICKEN,2.0,0,0,0.000,1.0,21.81,24.646138,4
523294848,2019-04-20 11:57:00,2019-04-20 12:40:43,00:43:43,2019-04-20 11:57:19,CHED&BEER.SOUP,2.0,0,0,0.000,1.0,7.98,9.017707,4
523294848,2019-04-20 11:57:00,2019-04-20 12:40:43,00:43:43,2019-04-20 11:57:19,HEINEKEN.Pt,1.0,3,2,0.562,1.0,9.33,10.543259,4
523294848,2019-04-20 11:57:00,2019-04-20 12:40:43,00:43:43,2019-04-20 11:57:19,POP.,4.0,0,0,0.000,1.0,3.99,4.508853,4
523294848,2019-04-20 11:57:00,2019-04-20 12:40:43,00:43:43,2019-04-20 11:57:19,FAJ.FOR.1,2.0,0,0,0.000,1.0,0.00,0.000000,4
523294848,2019-04-20 11:57:00,2019-04-20 12:40:43,00:43:43,2019-04-20 12:14:52,KIDS.,2.0,0,0,0.000,1.0,0.00,0.000000,4
523294848,2019-04-20 11:57:00,2019-04-20 12:40:43,00:43:43,2019-04-20 12:14:52,K.COOKIE,2.0,0,0,0.000,1.0,0.00,0.000000,4
523294848,2019-04-20 11:57:00,2019-04-20 12:40:43,00:43:43,2019-04-20 12:14:52,K.BUTTER.PASTA,2.0,0,0,0.000,1.0,7.81,8.825600,4
523294848,2019-04-20 11:57:00,2019-04-20 12:40:43,00:43:43,2019-04-20 12:14:52,KIDS.,2.0,0,0,0.000,1.0,0.00,0.000000,4


April 20th (Saturday), 11:57 - 12:40, 4 guests.

Meal Steps Description:
1. Start
    - 2 Dishes:
        - Flavored Chicken (for 1)
        - Cheddar & Beer Soup
    - 2 Drinks:
        - Heineken PINT
        - Pop
2. 15 Minutes Later:
    - Dishes:
        - Kids Pasta Butter
        - Kids Burger
        - Italian Burger
        - Broccoli Bites

`Lunch` / `Kids` / `Family Event`

#### 3. Table 514583822

In [26]:
hocky[hocky.index == 514583822].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
514583822,2019-04-13 19:46:00,2019-04-13 21:33:29,01:47:29,2019-04-13 19:46:54,BUDWEISER.Jg,1.0,71,2,1.538,1.0,21.72,24.543706,4
514583822,2019-04-13 19:46:00,2019-04-13 21:33:29,01:47:29,2019-04-13 19:47:27,QUESADILLAS,2.0,0,0,0.000,1.0,14.97,16.916173,4
514583822,2019-04-13 19:46:00,2019-04-13 21:33:29,01:47:29,2019-04-13 19:47:51,WINGS,2.0,0,0,0.000,1.0,0.00,0.000000,4
514583822,2019-04-13 19:46:00,2019-04-13 21:33:29,01:47:29,2019-04-13 19:47:51,WINGS.15PC,2.0,0,0,0.000,1.0,23.65,26.724615,4
514583822,2019-04-13 19:46:00,2019-04-13 21:33:29,01:47:29,2019-04-13 20:15:38,BUDWEISER.Jg,1.0,71,2,1.538,1.0,21.72,24.543706,4
514583822,2019-04-13 19:46:00,2019-04-13 21:33:29,01:47:29,2019-04-13 20:23:22,QUESADILLAS,2.0,0,0,0.000,1.0,14.97,16.916173,4
514583822,2019-04-13 19:46:00,2019-04-13 21:33:29,01:47:29,2019-04-13 20:23:29,CHK.PARMIGIANA,2.0,0,0,0.000,1.0,20.87,23.583202,4
514583822,2019-04-13 19:46:00,2019-04-13 21:33:29,01:47:29,2019-04-13 20:35:20,w.hot sauce,2.0,0,0,0.000,1.0,0.00,0.000000,4
514583822,2019-04-13 19:46:00,2019-04-13 21:33:29,01:47:29,2019-04-13 20:35:20,n/c.sauce,2.0,0,0,0.000,1.0,0.00,0.000000,4


April 13th (Saturday), 19:46 - 21:33, 4 guests.

Meal Steps Description:
1. Start
    - 2 Dishes:
        - Quesadillas
        - Wings
    - 1 Drink:
        - Budweiser
2. Half an Hour Later:
    - 1 Drink:
        - Budweiser
3. 8 Minutes Later:
    - 2 Dishes:
        - Quesadillas
        - Chicken Parmigiana
4. Half an Hour Later:
    - 1 Drink:
        - Pop
5. 15 Minutes Later:
    - 1 Drink:
        - Budweiser

`Social Gathering` / `Dinner`

#### 4. Table 524421347

In [27]:
hocky[hocky.index == 524421347].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
524421347,2019-04-21 17:31:00,2019-04-21 18:39:40,01:08:40,2019-04-21 17:31:04,WINGS.10PC,2.0,0,0,0.0,1.0,16.99,19.198043,3
524421347,2019-04-21 17:31:00,2019-04-21 18:39:40,01:08:40,2019-04-21 17:31:04,KIDPOP,4.0,0,0,0.0,1.0,0.00,0.000000,3
524421347,2019-04-21 17:31:00,2019-04-21 18:39:40,01:08:40,2019-04-21 17:31:04,Add.Poutine^,2.0,0,0,0.0,1.0,5.94,6.711970,3
524421347,2019-04-21 17:31:00,2019-04-21 18:39:40,01:08:40,2019-04-21 17:31:04,JACKS.VEG.NACHOS,2.0,0,0,0.0,1.0,21.26,24.022978,3
524421347,2019-04-21 17:31:00,2019-04-21 18:39:40,01:08:40,2019-04-21 17:31:04,WINGS,2.0,0,0,0.0,1.0,0.00,0.000000,3
524421347,2019-04-21 17:31:00,2019-04-21 18:39:40,01:08:40,2019-04-21 17:31:04,Add.Fries^,2.0,0,0,0.0,1.0,2.97,3.355985,3
524421347,2019-04-21 17:31:00,2019-04-21 18:39:40,01:08:40,2019-04-21 17:31:04,WINGS,2.0,0,0,0.0,1.0,0.00,0.000000,3
524421347,2019-04-21 17:31:00,2019-04-21 18:39:40,01:08:40,2019-04-21 17:31:04,KIDPOP,4.0,0,0,0.0,1.0,0.00,0.000000,3
524421347,2019-04-21 17:31:00,2019-04-21 18:39:40,01:08:40,2019-04-21 17:31:04,WINGS.10PC,2.0,0,0,0.0,1.0,16.99,19.198043,3


April 21th (Sunday), 17:31 - 18:39, 3 guests.

Meal Steps Description:
1. Start:
    - Dishes:
        - Wings + Poutine
        - Veggie Nachos
        - Wings + Fries
    - 1 Drink:
        - Mint Soda
2. Half an Hour Later:
    - 2 Dishes:
        - Poutine
        - Chicken Fingers

`Social Gathering` / `After Work`

#### 5. Table 521095118

In [28]:
hocky[hocky.index == 521095118].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
521095118,2019-04-17 21:20:00,2019-04-18 00:39:38,03:19:38,2019-04-17 21:18:00,JAMESON,3.0,0,0,0.000,1.0,6.00,6.780042,4
521095118,2019-04-17 21:20:00,2019-04-18 00:39:38,03:19:38,2019-04-17 21:18:00,JAMESON,3.0,0,0,0.000,1.0,6.00,6.780042,4
521095118,2019-04-17 21:20:00,2019-04-18 00:39:38,03:19:38,2019-04-17 21:18:00,PATRON SLVR,3.0,0,0,0.000,1.0,13.07,14.769192,4
521095118,2019-04-17 21:20:00,2019-04-18 00:39:38,03:19:38,2019-04-17 21:18:00,LG COORS LIGHT,1.0,280,2,0.591,1.0,7.25,8.192551,4
521095118,2019-04-17 21:20:00,2019-04-18 00:39:38,03:19:38,2019-04-17 21:18:00,JAMESON,3.0,0,0,0.000,1.0,6.00,6.780042,4
521095118,2019-04-17 21:20:00,2019-04-18 00:39:38,03:19:38,2019-04-17 21:18:00,LRG BANQUET,1.0,1120,2,0.591,1.0,7.75,8.757555,4
521095118,2019-04-17 21:20:00,2019-04-18 00:39:38,03:19:38,2019-04-17 21:18:00,LRG RICKARDS,1.0,191,2,0.591,1.0,7.75,8.757555,4
521095118,2019-04-17 21:20:00,2019-04-18 00:39:38,03:19:38,2019-04-17 21:18:00,PATRON SLVR,3.0,0,0,0.000,1.0,13.07,14.769192,4
521095118,2019-04-17 21:20:00,2019-04-18 00:39:38,03:19:38,2019-04-17 21:18:00,PATRON SLVR,3.0,0,0,0.000,1.0,13.07,14.769192,4


April 21th (Sunday), 21:20 - 00:40, 4 guests.

Meal Steps Description:
1. Start:
    - Beers:
        - Coors Light x 4
        - Large Banquet
        - Large Rickards
        - Large Bud Light
    - Alcohol:
        - Jameson x 3
        - Patron Silver Tequila x 4
        - Captain Morgan
        - Crown Royal x 4
    - Wine:
        - Red Sangria
2. 20 Minutes Later:
    - Beers:
        - Bud Light
    - Alcohol:
        - Jameson x 4
        - Lamb's Rum

`Drinking`

#### 6. Table 522574215

In [29]:
hocky[hocky.index == 522574215].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
522574215,2019-04-19 17:18:05,2019-04-19 18:17:19,00:59:14,2019-04-19 17:19:13,K- Chic Fingers,2.0,0,0,0.0,1.0,8.99,10.158057,4
522574215,2019-04-19 17:18:05,2019-04-19 18:17:19,00:59:14,2019-04-19 17:19:13,ED 6oz SteakFrit,2.0,0,0,0.0,1.0,11.99,13.547843,4
522574215,2019-04-19 17:18:05,2019-04-19 18:17:19,00:59:14,2019-04-19 17:19:13,K- Ling Pasta,2.0,0,0,0.0,1.0,8.99,10.158057,4
522574215,2019-04-19 17:18:05,2019-04-19 18:17:19,00:59:14,2019-04-19 17:19:13,Super Berry Kale,2.0,0,0,0.0,1.0,18.98,21.446043,4


April 19th (Friday), 17:18 - 18:17, 4 guests.

Meal Steps Description:
1. Start:
    - 4 Dishes:
        - Kids Chicken Fingers
        - Kids Pasta
        - Steak & Fries
        - Kale Salad

`Family Event` / `Kids`

#### 7. Table 522779155

In [30]:
hocky[hocky.index == 522779155].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
522779155,2019-04-20 00:47:00,2019-04-20 01:17:53,00:30:53,2019-04-20 00:47:00,HEINEKEN,1.0,3,1,0.33,1.0,7.20,8.136950,4
522779155,2019-04-20 00:47:00,2019-04-20 01:17:53,00:30:53,2019-04-20 00:47:00,WISER SPL BL,5.0,0,0,0.00,1.0,5.79,6.543464,4
522779155,2019-04-20 00:47:00,2019-04-20 01:17:53,00:30:53,2019-04-20 01:00:00,WISER SPL BL,5.0,0,0,0.00,1.0,5.79,6.543464,4
522779155,2019-04-20 00:47:00,2019-04-20 01:17:53,00:30:53,2019-04-20 01:00:00,GREY GOOSE,3.0,0,0,0.00,1.0,8.50,9.606122,4


July 20th (Saturday), 00:47 - 01:17, 4 guests.

1. Start:
    - 2 Side Dishes (WISER SPL BL)
2. 13 Minutes Later:
    - 2 Drinks:
        - Heineken
        - Grey Goose

`Drinking` / `???`

#### 8. Table 515363895

In [31]:
hocky[hocky.index == 515363895].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
515363895,2019-04-14 10:52:19,2019-04-14 11:53:30,01:01:11,2019-04-14 10:53:24,PB Big Breakfast,2.0,0,0,0.0,1.0,14.99,16.937782,3
515363895,2019-04-14 10:52:19,2019-04-14 11:53:30,01:01:11,2019-04-14 10:53:24,Fount Pop,4.0,0,0,0.0,1.0,2.99,3.378517,3
515363895,2019-04-14 10:52:19,2019-04-14 11:53:30,01:01:11,2019-04-14 10:53:24,PB French Toast,2.0,0,0,0.0,1.0,10.99,12.418027,3
515363895,2019-04-14 10:52:19,2019-04-14 11:53:30,01:01:11,2019-04-14 10:53:24,PB French Toast,2.0,0,0,0.0,1.0,12.98,14.666605,3
515363895,2019-04-14 10:52:19,2019-04-14 11:53:30,01:01:11,2019-04-14 10:53:24,Coffee,4.0,0,0,0.0,1.0,2.69,3.039535,3
515363895,2019-04-14 10:52:19,2019-04-14 11:53:30,01:01:11,2019-04-14 10:53:24,Coffee,4.0,0,0,0.0,1.0,2.69,3.039535,3


July 14th (Sunday), 10:52 - 11:53, 3 guests.

1. Start:
    - 3 Dishes:
        - Breakfast
        - French Toast x 2
    - 3 Drinks:
        - Fount pop
        - Coffee x 2

`Social Gathering`

#### 9. Table 519501788

In [32]:
hocky[hocky.index == 519501788].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
519501788,2019-04-15 18:29:00,2019-04-15 19:22:30,00:53:30,2019-04-15 18:29:42,COORS.LIGHT.Pt,1.0,280,2,0.473,1.0,7.62,8.610963,4
519501788,2019-04-15 18:29:00,2019-04-15 19:22:30,00:53:30,2019-04-15 18:29:42,KIDPOP,4.0,0,0,0.000,1.0,0.00,0.000000,4
519501788,2019-04-15 18:29:00,2019-04-15 19:22:30,00:53:30,2019-04-15 18:29:42,KIDPOP,4.0,0,0,0.000,1.0,0.00,0.000000,4
519501788,2019-04-15 18:29:00,2019-04-15 19:22:30,00:53:30,2019-04-15 18:41:57,KIDS.,2.0,0,0,0.000,1.0,0.00,0.000000,4
519501788,2019-04-15 18:29:00,2019-04-15 19:22:30,00:53:30,2019-04-15 18:41:57,TRIPLE.BAC.CHSBRG,2.0,0,0,0.000,1.0,20.97,23.697099,4
519501788,2019-04-15 18:29:00,2019-04-15 19:22:30,00:53:30,2019-04-15 18:41:57,DBL.CHSBURG,2.0,0,0,0.000,1.0,16.68,18.849195,4
519501788,2019-04-15 18:29:00,2019-04-15 19:22:30,00:53:30,2019-04-15 18:41:57,DBL.BAC.CHSBRG,2.0,0,0,0.000,1.0,17.87,20.193952,4
519501788,2019-04-15 18:29:00,2019-04-15 19:22:30,00:53:30,2019-04-15 18:41:57,K.BURGER,2.0,0,0,0.000,1.0,8.98,10.147828,4
519501788,2019-04-15 18:29:00,2019-04-15 19:22:30,00:53:30,2019-04-15 18:55:28,COORS.LIGHT.Pt,1.0,280,2,0.473,1.0,7.62,8.610963,4


July 15th (Monday), 18:29 - 19:22, 4 guests.

1. Start:
    - 2 Beers:
        - Coors Light PINT x 2
    - 4 Dishes:
        - Cheese Burger x 3
        - Kids Burger

`Family Event`

#### 10. Table 514529424

In [33]:
hocky[hocky.index == 514529424].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
514529424,2019-04-13 17:55:00,2019-04-13 19:07:57,01:12:57,2019-04-13 17:55:00,CAESAR VODKA DBL,3.0,0,0,0.0,1.0,10.50,11.863158,3
514529424,2019-04-13 17:55:00,2019-04-13 19:07:57,01:12:57,2019-04-13 17:55:00,CAESAR VODKA DBL,3.0,0,0,0.0,1.0,10.50,11.863158,3
514529424,2019-04-13 17:55:00,2019-04-13 19:07:57,01:12:57,2019-04-13 18:05:00,POTATO SKINS,2.0,0,0,0.0,1.0,5.95,6.722456,3
514529424,2019-04-13 17:55:00,2019-04-13 19:07:57,01:12:57,2019-04-13 18:05:00,CALAMARI,2.0,0,0,0.0,1.0,5.95,6.722456,3
514529424,2019-04-13 17:55:00,2019-04-13 19:07:57,01:12:57,2019-04-13 18:05:00,SOFT TACO (BEEF),2.0,0,0,0.0,1.0,5.95,6.722456,3
514529424,2019-04-13 17:55:00,2019-04-13 19:07:57,01:12:57,2019-04-13 18:24:00,CAESAR VODKA DBL,3.0,0,0,0.0,1.0,10.50,11.863158,3
514529424,2019-04-13 17:55:00,2019-04-13 19:07:57,01:12:57,2019-04-13 18:34:00,CAESAR VODKA DBL,3.0,0,0,0.0,1.0,10.50,11.863158,3


July 13th (Saturday), 17:55 - 19:07, 3 guests.

1. Start:
    - 3 Dishes:
        - Potato Skins
        - Calamari
        - Soft Taco
    - 4 Drinks:
        - Caesar Vodka x 4

`After Work` / `Social Gathering`

### Valentine

Pick 10 tables of 3 to 5 guests of **Valentine**.

In [34]:
valentine_tables_3to5 = get_tables_3to5(valentine_tables)
print("We have {} tables of 3 to 5 guests".format(len(valentine_tables_3to5)))

We have 4859 tables of 3 to 5 guests


In [35]:
# picked_valentine_tables = random.sample(list(valentine_tables_3to5.index), COUNT)

In [36]:
picked_valentine_tables = [
    447441456,
    435005397,
    435011399,
    447381509,
    446744761,
    447079489,
    434609975,
    448041560,
    447074978,
    434603220
]

In [37]:
print("Got {} tables".format(len(picked_valentine_tables)))

Got 10 tables


#### 1. Table 447441456

In [62]:
valentine[valentine.index == 447441456].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
447441456,2019-02-14 17:06:00,2019-02-14 18:27:42,01:21:42,2019-02-14 17:06:00,BANQUET.XL,1.0,1120,2,0.591,1.0,10.01,11.309856,3
447441456,2019-02-14 17:06:00,2019-02-14 18:27:42,01:21:42,2019-02-14 17:13:00,BRUSCHETTA.,2.0,0,0,0.000,1.0,13.31,15.038380,3
447441456,2019-02-14 17:06:00,2019-02-14 18:27:42,01:21:42,2019-02-14 17:15:00,$Sub.Swt.Pot.Fry-,2.0,0,0,0.000,1.0,2.97,3.355671,3
447441456,2019-02-14 17:06:00,2019-02-14 18:27:42,01:21:42,2019-02-14 17:15:00,K.WINGS,2.0,0,0,0.000,1.0,8.98,10.146104,3
447441456,2019-02-14 17:06:00,2019-02-14 18:27:42,01:21:42,2019-02-14 17:15:00,FAJ.BOWL.CHK,2.0,0,0,0.000,1.0,19.99,22.585816,3
447441456,2019-02-14 17:06:00,2019-02-14 18:27:42,01:21:42,2019-02-14 17:15:00,KIDS.,2.0,0,0,0.000,1.0,0.00,0.000000,3
447441456,2019-02-14 17:06:00,2019-02-14 18:27:42,01:21:42,2019-02-14 17:15:00,KIDS.,2.0,0,0,0.000,1.0,0.00,0.000000,3
447441456,2019-02-14 17:06:00,2019-02-14 18:27:42,01:21:42,2019-02-14 17:15:00,K.PASTA CHS,2.0,0,0,0.000,1.0,7.81,8.824173,3
447441456,2019-02-14 17:06:00,2019-02-14 18:27:42,01:21:42,2019-02-14 17:43:00,KIDPOP,4.0,0,0,0.000,1.0,0.00,0.000000,3


February 14th (Thursday), 17:06 - 18:27, 3 guests.

1. Start:
    - 1 Beer:
        - Banquet
2. 10 Minutes Later:
    - Dishes:
        - Bruschetta
        - Chicken
        - Kids Pasta Cheese
        - Kids Wings

`Family Event`

#### 2. Table 435005397

In [63]:
valentine[valentine.index == 435005397].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
435005397,2019-02-11 12:05:00,2019-02-11 13:56:33,01:51:33,2019-02-11 12:05:00,LG GUINNESS,1.0,26,2,0.591,1.0,8.75,9.887636,3
435005397,2019-02-11 12:05:00,2019-02-11 13:56:33,01:51:33,2019-02-11 12:05:00,LRG BANQUET,1.0,1120,2,0.591,1.0,7.75,8.757621,3
435005397,2019-02-11 12:05:00,2019-02-11 13:56:33,01:51:33,2019-02-11 12:17:00,SPRING ROLLS,2.0,0,0,0.000,1.0,12.00,13.560187,3
435005397,2019-02-11 12:05:00,2019-02-11 13:56:33,01:51:33,2019-02-11 12:20:00,CHICKEN BITES,2.0,0,0,0.000,1.0,14.00,15.820218,3
435005397,2019-02-11 12:05:00,2019-02-11 13:56:33,01:51:33,2019-02-11 12:24:00,ICE TEA,4.0,0,0,0.000,1.0,3.25,3.672551,3
435005397,2019-02-11 12:05:00,2019-02-11 13:56:33,01:51:33,2019-02-11 12:34:00,LG GUINNESS,1.0,26,2,0.591,1.0,8.75,9.887636,3
435005397,2019-02-11 12:05:00,2019-02-11 13:56:33,01:51:33,2019-02-11 12:34:00,LRG BANQUET,1.0,1120,2,0.591,1.0,7.75,8.757621,3
435005397,2019-02-11 12:05:00,2019-02-11 13:56:33,01:51:33,2019-02-11 12:40:00,NACHOS SUPREME,2.0,0,0,0.000,1.0,16.00,18.080249,3
435005397,2019-02-11 12:05:00,2019-02-11 13:56:33,01:51:33,2019-02-11 12:40:00,NEW YORK 16 OZ,2.0,0,0,0.000,1.0,32.00,36.160498,3


February 11th (Monday), 12:05 - 13:56, 3 guests.

1. Start:
    - 2 Beers:
        - Large Guinness
        - Large Banquet
2. 15 Minutes Later:
    - 2 Dishes:
        - Spring Rolls
        - Chicken Bites
    - 1 Drink:
        - Ice Tea
3. 10 Minutes Later:
    - 2 Beers:
        - Large Guinness
        - Large Banquet
    - 3 Dishes:
        - Nachos
        - Burger x 2
4. 40 Minutes Later:
    - 2 Beers:
        - Large Guinness
        - Large Banquet

`Social Gathering`

#### 3. Table 435011399

In [64]:
valentine[valentine.index == 435011399].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
435011399,2019-02-11 13:05:24,2019-02-11 14:01:59,00:56:35,2019-02-11 13:06:33,Create Omelette,2.0,0,0,0.0,1.0,11.69,13.379721,3
435011399,2019-02-11 13:05:24,2019-02-11 14:01:59,00:56:35,2019-02-11 13:06:33,*No Bev,5.0,0,0,0.0,1.0,0.00,0.000000,3
435011399,2019-02-11 13:05:24,2019-02-11 14:01:59,00:56:35,2019-02-11 13:06:33,Freddy Tuna Sal,2.0,0,0,0.0,1.0,11.69,13.379721,3
435011399,2019-02-11 13:05:24,2019-02-11 14:01:59,00:56:35,2019-02-11 13:06:33,Freddy Tuna Sal,2.0,0,0,0.0,1.0,11.69,13.379721,3
435011399,2019-02-11 13:05:24,2019-02-11 14:01:59,00:56:35,2019-02-11 13:56:14,Coffee,4.0,0,0,0.0,1.0,2.43,2.781242,3
435011399,2019-02-11 13:05:24,2019-02-11 14:01:59,00:56:35,2019-02-11 13:56:14,Coffee,4.0,0,0,0.0,1.0,2.42,2.769797,3
435011399,2019-02-11 13:05:24,2019-02-11 14:01:59,00:56:35,2019-02-11 13:56:14,Coffee Decaf,4.0,0,0,0.0,1.0,2.42,2.769797,3


February 11th (Monday), 13:05 - 14:02, 3 guests.

1. Start:
    - 3 Dishes:
        - Omelette
        - Freddy Tuna Salad
2. 50 Minutes Later:
    - 3 Drinks:
        - Coffee x 3

`Social Gathering`

#### 4. Table 447381509

In [65]:
valentine[valentine.index == 447381509].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
447381509,2019-02-14 13:56:36,2019-02-14 14:38:53,00:42:17,2019-02-14 13:56:59,8oz Lucky SHRZ,6.0,0,0,-0.0,-1.0,0.00,0.000000,3
447381509,2019-02-14 13:56:36,2019-02-14 14:38:53,00:42:17,2019-02-14 13:56:59,Fount Pop,4.0,0,0,0.0,1.0,2.99,3.378595,3
447381509,2019-02-14 13:56:36,2019-02-14 14:38:53,00:42:17,2019-02-14 13:57:44,Crispy Calamari,2.0,0,0,0.0,1.0,13.99,15.808210,3
447381509,2019-02-14 13:56:36,2019-02-14 14:38:53,00:42:17,2019-02-14 13:57:44,Maple Nut Salmon,2.0,0,0,0.0,1.0,24.99,28.237825,3
447381509,2019-02-14 13:56:36,2019-02-14 14:38:53,00:42:17,2019-02-14 13:57:44,10oz Steak Frite,2.0,0,0,0.0,1.0,29.99,33.887650,3
447381509,2019-02-14 13:56:36,2019-02-14 14:38:53,00:42:17,2019-02-14 13:57:44,Pete's Deli,2.0,0,0,0.0,1.0,16.99,19.198105,3
447381509,2019-02-14 13:56:36,2019-02-14 14:38:53,00:42:17,2019-02-14 13:57:44,Pete's Deli,2.0,0,0,-0.0,-1.0,0.00,0.000000,3
447381509,2019-02-14 13:56:36,2019-02-14 14:38:53,00:42:17,2019-02-14 13:59:57,8oz Deakin SHZ,6.0,0,0,0.0,1.0,11.00,12.429615,3


**???**

February 14th (Thursday), 13:56 - 14:38, 3 guests.

1. Start:
    - 3 (/4 ?) Dishes:
        - Crispy Calamari
        - Maple Nut Salmon
        - Steak Frite
        - Deli (?)
    - 1 Wine:
        - Shiraz

`Lunch`

#### 5. Table 446744761

In [66]:
valentine[valentine.index == 446744761].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
446744761,2019-02-12 11:15:00,2019-02-12 12:57:53,01:42:53,2019-02-12 11:15:00,BADOIT.750ML,4.0,0,0,0.0,1.0,6.56,7.412722,3
446744761,2019-02-12 11:15:00,2019-02-12 12:57:53,01:42:53,2019-02-12 11:25:00,BRUSCHETTA.,2.0,0,0,0.0,1.0,13.31,15.040142,3
446744761,2019-02-12 11:15:00,2019-02-12 12:57:53,01:42:53,2019-02-12 11:25:00,LOBSTER.CRB.DIP,2.0,0,0,0.0,1.0,14.97,16.915922,3
446744761,2019-02-12 11:15:00,2019-02-12 12:57:53,01:42:53,2019-02-12 11:25:00,CALAMARI,2.0,0,0,0.0,1.0,14.35,16.215329,3
446744761,2019-02-12 11:15:00,2019-02-12 12:57:53,01:42:53,2019-02-12 11:35:00,BOMBAY.CHICKEN,2.0,0,0,0.0,1.0,19.97,22.565863,3
446744761,2019-02-12 11:15:00,2019-02-12 12:57:53,01:42:53,2019-02-12 11:35:00,TUNA.SALAD+,2.0,0,0,0.0,1.0,19.87,22.452864,3
446744761,2019-02-12 11:15:00,2019-02-12 12:57:53,01:42:53,2019-02-12 11:35:00,TUNA.BOWL,2.0,0,0,0.0,1.0,20.77,23.469853,3
446744761,2019-02-12 11:15:00,2019-02-12 12:57:53,01:42:53,2019-02-12 11:54:00,POP.,4.0,0,0,0.0,1.0,3.99,4.508653,3
446744761,2019-02-12 11:15:00,2019-02-12 12:57:53,01:42:53,2019-02-12 11:54:00,POP.,4.0,0,0,0.0,1.0,3.99,4.508653,3


February 12th (Tuesday), 11:15 - 12:57, 3 guests.

1. Start:
    - 1 Drink:
        - Badot Water
2. 10 Minutes Later:
    - 3 Dishes:
        - Bruschetta
        - Lobster
        - Calamari
3. 10 Minutes Later:
    - 3 Dishes:
        - Chicken
        - Tuna Salad
        - Tuna
4. 20 Minutes Later:
    - 2 Drinks:
        - Pop x 2

`Lunch`

#### 6. Table 447079489

In [67]:
valentine[valentine.index == 447079489].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
447079489,2019-02-13 18:25:29,2019-02-13 19:14:24,00:48:55,2019-02-13 18:25:48,Maple Nut Salmon,2.0,0,0,0.0,1.0,19.99,22.588469,4
447079489,2019-02-13 18:25:29,2019-02-13 19:14:24,00:48:55,2019-02-13 18:25:48,Grill Cheeseburg,2.0,0,0,0.0,1.0,15.99,18.068515,4
447079489,2019-02-13 18:25:29,2019-02-13 19:14:24,00:48:55,2019-02-13 18:25:48,*Water,4.0,0,0,0.0,1.0,0.00,0.000000,4
447079489,2019-02-13 18:25:29,2019-02-13 19:14:24,00:48:55,2019-02-13 18:25:48,Chicken Parmesan,2.0,0,0,0.0,1.0,18.99,21.458481,4
447079489,2019-02-13 18:25:29,2019-02-13 19:14:24,00:48:55,2019-02-13 18:25:48,Fish & Chips,2.0,0,0,0.0,1.0,13.99,15.808538,4
447079489,2019-02-13 18:25:29,2019-02-13 19:14:24,00:48:55,2019-02-13 19:11:01,Fount Pop,4.0,0,0,0.0,3.0,8.97,10.135996,4


February 13th (Wednesday), 18:25 - 19:14, 4 guests.

1. Start:
    - 4 Dishes:
        - Maple Nut Salmon
        - Grill Cheeseburger
        - Chicken Parmesan
        - Fish & Chips
    - 1 Drink:
        - 1 Water
2. 45 Minutes Later:
    - 1 Drink:
        - 1 Pop

`After Work`

#### 7. Table 434609975

In [68]:
valentine[valentine.index == 434609975].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
434609975,2019-02-10 13:12:45,2019-02-10 14:17:33,01:04:48,2019-02-10 13:13:32,8oz Drifting CAB,6.0,0,0,0.0,1.0,12.00,13.559475,3
434609975,2019-02-10 13:12:45,2019-02-10 14:17:33,01:04:48,2019-02-10 13:13:32,8oz Brancott SB,6.0,0,0,0.0,1.0,12.00,13.559475,3
434609975,2019-02-10 13:12:45,2019-02-10 14:17:33,01:04:48,2019-02-10 13:13:39,S-Pork Potstickr,2.0,0,0,0.0,1.0,5.99,6.768438,3
434609975,2019-02-10 13:12:45,2019-02-10 14:17:33,01:04:48,2019-02-10 13:13:48,K- Chic Fingers,2.0,0,0,0.0,1.0,11.98,13.536876,3
434609975,2019-02-10 13:12:45,2019-02-10 14:17:33,01:04:48,2019-02-10 13:20:30,Salmon Teriyaki,2.0,0,0,0.0,1.0,19.99,22.587825,3
434609975,2019-02-10 13:12:45,2019-02-10 14:17:33,01:04:48,2019-02-10 13:20:30,Tuna Poke Bowl,2.0,0,0,0.0,1.0,17.99,20.327912,3


February 10th (Sunday), 13:12 - 14:17, 3 guests.

1. Start:
    - 4 Dishes:
        - Potstickr
        - Kids Chicken Fingers
        - Salmon
        - Tuna
    - 2 Wines:
        - Drifting Cabernet Sauvignon
        - Brancott

`Lunch`

#### 8. Table 448041560

In [69]:
valentine[valentine.index == 448041560].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
448041560,2019-02-15 19:39:25,2019-02-15 20:44:38,01:05:13,2019-02-15 19:40:42,EXTRA Egg,2.0,0,0,0.000,1.0,1.99,2.248707,3
448041560,2019-02-15 19:39:25,2019-02-15 20:44:38,01:05:13,2019-02-15 19:40:42,Canadian,1.0,183,1,0.341,1.0,5.50,6.215018,3
448041560,2019-02-15 19:39:25,2019-02-15 20:44:38,01:05:13,2019-02-15 19:40:42,Freddy Tuna Sal,2.0,0,0,0.000,1.0,12.99,14.678744,3
448041560,2019-02-15 19:39:25,2019-02-15 20:44:38,01:05:13,2019-02-15 19:40:42,Maple Nut Salmon,2.0,0,0,0.000,1.0,24.99,28.238784,3
448041560,2019-02-15 19:39:25,2019-02-15 20:44:38,01:05:13,2019-02-15 19:40:42,Meatloaf,2.0,0,0,0.000,1.0,13.99,15.808747,3


February 15th (Friday), 19:39 - 20:44, 3 guests.

1. Start:
    - 3 Dishes:
        - Tuna Salad
        - Maple Nut Salmon
        - Meatloaf
    - 1 Drink:
        - Canadian Beer

`Dinner`

#### 9. Table 447074978

In [70]:
valentine[valentine.index == 447074978].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
447074978,2019-02-13 17:57:00,2019-02-13 19:25:12,01:28:12,2019-02-13 17:57:00,Add.Plain.Mash--,2.0,0,0,0.000,1.0,0.00,0.000000,4
447074978,2019-02-13 17:57:00,2019-02-13 19:25:12,01:28:12,2019-02-13 17:57:00,QUESA.STACK,2.0,0,0,0.000,1.0,18.59,21.007192,4
447074978,2019-02-13 17:57:00,2019-02-13 19:25:12,01:28:12,2019-02-13 17:57:00,Add.Plain.Mash--,2.0,0,0,0.000,1.0,0.00,0.000000,4
447074978,2019-02-13 17:57:00,2019-02-13 19:25:12,01:28:12,2019-02-13 17:57:00,FINGERS.SAUCED,2.0,0,0,0.000,1.0,16.87,19.063547,4
447074978,2019-02-13 17:57:00,2019-02-13 19:25:12,01:28:12,2019-02-13 17:57:00,CALAMARI,2.0,0,0,0.000,1.0,14.35,16.215880,4
447074978,2019-02-13 17:57:00,2019-02-13 19:25:12,01:28:12,2019-02-13 17:57:00,FINGERS.SAUCED,2.0,0,0,0.000,1.0,16.87,19.063547,4
447074978,2019-02-13 17:57:00,2019-02-13 19:25:12,01:28:12,2019-02-13 17:57:00,CAULI.BITES+,2.0,0,0,0.000,1.0,13.38,15.119754,4
447074978,2019-02-13 17:57:00,2019-02-13 19:25:12,01:28:12,2019-02-13 17:57:00,FINGERS.SAUCED,2.0,0,0,0.000,1.0,16.87,19.063547,4
447074978,2019-02-13 17:57:00,2019-02-13 19:25:12,01:28:12,2019-02-13 17:57:00,SUB.PARM.POUT,2.0,0,0,0.000,1.0,5.97,6.746258,4


February 13th (Wednesday), 17:57 - 19:25, 4 guests.

1. Start:
    - 4 Dishes + 3 Sharables:
        - Quesadilla
        - Chicken Fingers x 3
        - Calamari
        - Cauliflower Bites
        - Poutine
2. 30 Minutes Later:
    - 3 Soft Drinks:
        - Water x 2
        - Pop
    - 1 Beer:
        - Lagunita PINT
3. 30 Minutes Later:
    - 1 Dish:
        - Cookie & Milk

`Dinner` / `After Work`

#### 10. Table 434603220

In [71]:
valentine[valentine.index == 434603220].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
434603220,2019-02-10 12:26:12,2019-02-10 13:47:54,01:21:42,2019-02-10 12:26:17,Coffee,4.0,0,0,0.0,1.0,2.469298,2.790400,3
434603220,2019-02-10 12:26:12,2019-02-10 13:47:54,01:21:42,2019-02-10 12:26:17,Coffee,4.0,0,0,0.0,1.0,2.469298,2.790400,3
434603220,2019-02-10 12:26:12,2019-02-10 13:47:54,01:21:42,2019-02-10 12:44:42,PB Big Breakfast,2.0,0,0,0.0,1.0,13.760140,15.549476,3
434603220,2019-02-10 12:26:12,2019-02-10 13:47:54,01:21:42,2019-02-10 12:44:42,Greek LG,2.0,0,0,0.0,1.0,20.626440,23.308653,3
434603220,2019-02-10 12:26:12,2019-02-10 13:47:54,01:21:42,2019-02-10 12:44:42,The B.B.B.L.T,2.0,0,0,0.0,1.0,13.760140,15.549476,3
434603220,2019-02-10 12:26:12,2019-02-10 13:47:54,01:21:42,2019-02-10 12:44:42,Fount Pop,4.0,0,0,0.0,1.0,2.744684,3.101597,3


February 10th (Sunday), 12:26 - 13:47, 3 guests.

1. Start:
    - 2 Drinks:
        - Coffee x 2
2. 20 Minutes Later:
    - 3 Dishes:
        - Breakfast
        - Greek
        - ???
    - 1 Drink:
        - Pop

`Lunch`

### Silvester

Pick 10 tables of 3 to 5 guests of **Silvester**.

In [48]:
silvester_tables_3to5 = get_tables_3to5(silvester_tables)
print("We have {} tables of 3 to 5 guests".format(len(silvester_tables_3to5)))

We have 14636 tables of 3 to 5 guests


In [49]:
# picked_silvester_tables = random.sample(list(silvester_tables_3to5.index), COUNT)

In [50]:
picked_silvester_tables = [
    362939931,
    361304992,
    363233153,
    360790021,
    362924064,
    361776660,
    361815160,
    360000762,
    360402335,
    361369624
]

In [51]:
print("Got {} tables".format(len(picked_silvester_tables)))

Got 10 tables


#### 1. Table 362939931

In [72]:
silvester[silvester.index == 362939931].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
362939931,2019-01-02 18:23:00,2019-01-02 19:59:38,01:36:38,2019-01-02 19:20:00,LG SMITHWICKS,1.0,720,2,0.591,1.0,8.75,9.886598,3
362939931,2019-01-02 18:23:00,2019-01-02 19:59:38,01:36:38,2019-01-02 19:20:00,BUFFALO CHIC WRAP,2.0,0,0,0.000,1.0,15.50,17.513402,3


January 2nd (Wednesday), 18:23 - 19:59, 3 guests.

1. Start:
    - 1 Dish:
        - 1 Buffalo Chicken
    - 1 Drink:
        - Smithwicks Beer

`After Work`

#### 2. Table 361304992

In [73]:
silvester[silvester.index == 361304992].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
361304992,2018-12-29 15:25:00,2018-12-29 16:13:18,00:48:18,2018-12-29 15:25:00,KIDPOP,4.0,0,0,0.000,1.0,0.00,0.000000,3
361304992,2018-12-29 15:25:00,2018-12-29 16:13:18,00:48:18,2018-12-29 15:25:00,ONTARIO.CRAFT.XL,1.0,2482,2,0.591,1.0,10.30,11.636786,3
361304992,2018-12-29 15:25:00,2018-12-29 16:13:18,00:48:18,2018-12-29 15:25:00,KIDBEV,4.0,0,0,0.000,1.0,0.00,0.000000,3
361304992,2018-12-29 15:25:00,2018-12-29 16:13:18,00:48:18,2018-12-29 15:29:00,n/c.sauce,2.0,0,0,0.000,1.0,0.00,0.000000,3
361304992,2018-12-29 15:25:00,2018-12-29 16:13:18,00:48:18,2018-12-29 15:29:00,garlic.aioli,2.0,0,0,0.000,1.0,0.00,0.000000,3
361304992,2018-12-29 15:25:00,2018-12-29 16:13:18,00:48:18,2018-12-29 15:29:00,$$Truffle.Fries$$,2.0,0,0,0.000,1.0,2.97,3.355462,3
361304992,2018-12-29 15:25:00,2018-12-29 16:13:18,00:48:18,2018-12-29 15:29:00,K.PIZZA,2.0,0,0,0.000,1.0,8.98,10.145470,3
361304992,2018-12-29 15:25:00,2018-12-29 16:13:18,00:48:18,2018-12-29 15:29:00,ITALIAN.BURGER**,2.0,0,0,0.000,1.0,17.62,19.906813,3
361304992,2018-12-29 15:25:00,2018-12-29 16:13:18,00:48:18,2018-12-29 15:29:00,KIDS.,2.0,0,0,0.000,1.0,0.00,0.000000,3


December 29th (Saturday), 15:25 - 16:13, 3 guests.

1. Start:
    - 4 Dishes:
        - Truffle Fries
        - Italian Burger
        - Kids Pizza x 2
    - Soft Drinks:
        - Kids Pop x 2
    - 1 Beer:
        - Ontario Craft

`Family Event`

#### 3. Table 363233153

In [74]:
silvester[silvester.index == 363233153].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
363233153,2019-01-03 12:41:00,2019-01-03 13:34:41,00:53:41,2019-01-03 12:41:00,ALABAMA SLAMMER,3.0,0,0,0.0,1.0,6.75,7.591907,5
363233153,2019-01-03 12:41:00,2019-01-03 13:34:41,00:53:41,2019-01-03 12:42:00,PENNE SUNDRIED,2.0,0,0,0.0,1.0,5.95,6.692125,5
363233153,2019-01-03 12:41:00,2019-01-03 13:34:41,00:53:41,2019-01-03 12:42:00,MAC 'N CHEESE,2.0,0,0,0.0,1.0,5.95,6.692125,5
363233153,2019-01-03 12:41:00,2019-01-03 13:34:41,00:53:41,2019-01-03 12:42:00,MAC 'N CHEESE,2.0,0,0,0.0,1.0,5.95,6.692125,5
363233153,2019-01-03 12:41:00,2019-01-03 13:34:41,00:53:41,2019-01-03 12:42:00,GRILLED CHEESE 3.0,2.0,0,0,0.0,1.0,5.95,6.692125,5
363233153,2019-01-03 12:41:00,2019-01-03 13:34:41,00:53:41,2019-01-03 12:42:00,VEGGIE BURGER,2.0,0,0,0.0,1.0,5.95,6.692125,5
363233153,2019-01-03 12:41:00,2019-01-03 13:34:41,00:53:41,2019-01-03 12:42:00,GRILLED CHEESE 3.0,2.0,0,0,0.0,1.0,7.90,8.885343,5
363233153,2019-01-03 12:41:00,2019-01-03 13:34:41,00:53:41,2019-01-03 12:42:00,VEGGIE BURGER,2.0,0,0,0.0,1.0,5.95,6.692125,5


January 3rd (Thursday), 12:41 - 13:34, 5 guests.

1. Start:
    - Dishes:
        - Pasta
        - Mac & Cheese x 2
        - Grilled Cheese x 2
        - Veggie Burger x 2
    - 1 Cocktail:
        - Alabama Slammer

`Lunch`

#### 4. Table 360790021

In [75]:
silvester[silvester.index == 360790021].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
360790021,2018-12-28 14:21:35,2018-12-28 14:57:45,00:36:10,2018-12-28 14:21:41,Killer Shake,4.0,0,0,0.0,1.0,4.99,5.638489,3
360790021,2018-12-28 14:21:35,2018-12-28 14:57:45,00:36:10,2018-12-28 14:26:14,PB Jumbo Club,2.0,0,0,0.0,1.0,14.48,16.361787,3
360790021,2018-12-28 14:21:35,2018-12-28 14:57:45,00:36:10,2018-12-28 14:26:14,SIDE Pickles,2.0,0,0,-0.0,-1.0,0.00,0.000000,3
360790021,2018-12-28 14:21:35,2018-12-28 14:57:45,00:36:10,2018-12-28 14:26:14,Pesto Chicken,2.0,0,0,0.0,1.0,18.99,21.457896,3
360790021,2018-12-28 14:21:35,2018-12-28 14:57:45,00:36:10,2018-12-28 14:26:14,Grilled Cheese,2.0,0,0,0.0,1.0,13.48,15.231829,3


December 28th (Friday), 14:21 - 14:57, 3 guests.

1. Start:
    - 3 Dishes:
        - Jumbo Club
        - Pesto Chicken
        - Grilled Cheese
    - 1 Soft Drink:
        - Killer Shake

`Lunch`

#### 5. Table 362924064

In [76]:
silvester[silvester.index == 362924064].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
362924064,2019-01-02 17:29:00,2019-01-02 18:45:58,01:16:58,2019-01-02 17:29:00,POP.,4.0,0,0,0.0,1.0,3.99,4.509349,3
362924064,2019-01-02 17:29:00,2019-01-02 18:45:58,01:16:58,2019-01-02 17:29:00,POP.,4.0,0,0,0.0,1.0,3.99,4.509349,3
362924064,2019-01-02 17:29:00,2019-01-02 18:45:58,01:16:58,2019-01-02 17:35:00,CHICKEN.FINGERS,2.0,0,0,0.0,1.0,15.99,18.071302,3
362924064,2019-01-02 17:29:00,2019-01-02 18:45:58,01:16:58,2019-01-02 17:35:00,TUNA.BOWL,2.0,0,0,0.0,1.0,0.00,0.000000,3
362924064,2019-01-02 17:29:00,2019-01-02 18:45:58,01:16:58,2019-01-02 17:35:00,Side.Caesar,2.0,0,0,0.0,1.0,0.00,0.000000,3
362924064,2019-01-02 17:29:00,2019-01-02 18:45:58,01:16:58,2019-01-02 17:56:00,SESAME CHICKEN,2.0,0,0,0.0,1.0,0.00,0.000000,3


January 2nd (Wednesday), 17:29 - 18:45, 3 guests.

1. Start:
    - 2 Soft Drinks:
        - Pop x 2
2. 5 Minutes Later:
    - 1 Dish:
        - Chicken Fingers

`After Work`

#### 6. Table 361776660

In [77]:
silvester[silvester.index == 361776660].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
361776660,2018-12-30 19:03:00,2018-12-30 20:27:17,01:24:17,2018-12-30 19:03:00,Side Caesar.,2.0,0,0,0.000,1.0,0.00,0.000000,4
361776660,2018-12-30 19:03:00,2018-12-30 20:27:17,01:24:17,2018-12-30 19:03:00,ASIAGO.BOWTIE,2.0,0,0,0.000,1.0,19.99,22.589221,4
361776660,2018-12-30 19:03:00,2018-12-30 20:27:17,01:24:17,2018-12-30 19:03:00,WINGS.5PC,2.0,0,0,0.000,1.0,10.47,11.831373,4
361776660,2018-12-30 19:03:00,2018-12-30 20:27:17,01:24:17,2018-12-30 19:03:00,FRIED.CHK.SAND,2.0,0,0,0.000,1.0,17.09,19.312145,4
361776660,2018-12-30 19:03:00,2018-12-30 20:27:17,01:24:17,2018-12-30 19:03:00,BREWSKY.RT.BEER,4.0,0,0,0.000,1.0,4.45,5.028616,4
361776660,2018-12-30 19:03:00,2018-12-30 20:27:17,01:24:17,2018-12-30 19:03:00,ADD CAESAR,2.0,0,0,0.000,1.0,5.25,5.932637,4
361776660,2018-12-30 19:03:00,2018-12-30 20:27:17,01:24:17,2018-12-30 19:03:00,WINGS..,2.0,0,0,0.000,1.0,0.00,0.000000,4
361776660,2018-12-30 19:03:00,2018-12-30 20:27:17,01:24:17,2018-12-30 19:03:00,CHICKEN.FINGERS,2.0,0,0,0.000,1.0,15.99,18.069117,4
361776660,2018-12-30 19:03:00,2018-12-30 20:27:17,01:24:17,2018-12-30 19:03:00,POP.,4.0,0,0,0.000,1.0,3.99,4.508804,4


December 30th (Sunday), 19:03 - 20:27, 4 guests.

1. Start:
    - Dishes:
        - Asiago & BowTie
        - Wingos
        - Fried Chicken
        - Chicken Fingers
    - Beers:
        - Brewsky Beer
        - Mill Street
    - Soft Drinks:
        - Pop
2. 20 Minutes Later:
    - 1 Cocktai
3. 40 Minutes Later:
    - 2 Desserts:
        - Ice Cream
        - Brownie Cookie

`Dinner`

#### 7. Table 361815160

In [78]:
silvester[silvester.index == 361815160].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
361815160,2018-12-30 20:14:00,2018-12-30 21:09:42,00:55:42,2018-12-30 20:14:58,PEROGIES,2.0,0,0,0.000,1.0,5.95,6.680803,5
361815160,2018-12-30 20:14:00,2018-12-30 21:09:42,00:55:42,2018-12-30 20:14:58,VEGGIE BURGER,2.0,0,0,0.000,1.0,7.90,8.870309,5
361815160,2018-12-30 20:14:00,2018-12-30 21:09:42,00:55:42,2018-12-30 20:14:58,CHICKEN SANDWICH,2.0,0,0,0.000,1.0,6.95,7.803627,5
361815160,2018-12-30 20:14:00,2018-12-30 21:09:42,00:55:42,2018-12-30 20:14:58,VIRGIN RASP MOJITO,4.0,0,0,0.000,1.0,3.75,4.210590,5
361815160,2018-12-30 20:14:00,2018-12-30 21:09:42,00:55:42,2018-12-30 20:14:58,OLD FASHIONED,3.0,0,0,0.000,1.0,12.25,13.754594,5
361815160,2018-12-30 20:14:00,2018-12-30 21:09:42,00:55:42,2018-12-30 20:40:42,BUDWEISER TAP,1.0,71,2,0.414,1.0,6.00,6.736944,5
361815160,2018-12-30 20:14:00,2018-12-30 21:09:42,00:55:42,2018-12-30 20:40:42,PEROGIES,2.0,0,0,0.000,1.0,5.95,6.680803,5
361815160,2018-12-30 20:14:00,2018-12-30 21:09:42,00:55:42,2018-12-30 21:00:28,COKE,4.0,0,0,0.000,1.0,2.95,3.312331,5


December 30th (Sunday), 20:14 - 21:09, 5 guests.

1. Start:
    - 3 Dishes:
        - Perogies
        - Veggie Burger
        - Chicken Sandwich
    - 1 Soft Drink:
        - Virgin Rasp Mojito
    - 1 Cocktail:
        - Old Fashioned
2. 25 Minutes Later:
    - 1 Beer:
        - Budweiser
    - 1 Dish:
        - Perogies
3. 20 Minutes Later:
    - 1 Soft Drink:
        - Coke

`Dinner`

#### 8. Table 360000762

In [79]:
silvester[silvester.index == 360000762].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
360000762,2018-12-26 16:30:50,2018-12-26 17:32:00,01:01:10,2018-12-26 16:31:30,ED Starter,2.0,0,0,0.000,1.0,0.0,0.00,4
360000762,2018-12-26 16:30:50,2018-12-26 17:32:00,01:01:10,2018-12-26 16:31:30,ED Salmon,2.0,0,0,0.000,1.0,18.0,20.34,4
360000762,2018-12-26 16:30:50,2018-12-26 17:32:00,01:01:10,2018-12-26 16:31:30,ED Starter,2.0,0,0,0.000,1.0,0.0,0.00,4
360000762,2018-12-26 16:30:50,2018-12-26 17:32:00,01:01:10,2018-12-26 16:31:30,ED Starter,2.0,0,0,0.000,1.0,0.0,0.00,4
360000762,2018-12-26 16:30:50,2018-12-26 17:32:00,01:01:10,2018-12-26 16:31:30,ED Starter,2.0,0,0,0.000,1.0,0.0,0.00,4
360000762,2018-12-26 16:30:50,2018-12-26 17:32:00,01:01:10,2018-12-26 16:31:30,ED Salmon,2.0,0,0,0.000,1.0,18.0,20.34,4
360000762,2018-12-26 16:30:50,2018-12-26 17:32:00,01:01:10,2018-12-26 16:31:30,ED Sirloin,2.0,0,0,0.000,1.0,18.0,20.34,4
360000762,2018-12-26 16:30:50,2018-12-26 17:32:00,01:01:10,2018-12-26 16:31:30,ED Turkey Bologn,2.0,0,0,0.000,1.0,18.0,20.34,4
360000762,2018-12-26 16:30:50,2018-12-26 17:32:00,01:01:10,2018-12-26 16:31:30,****COURSE 2****,2.0,0,0,0.000,1.0,0.0,0.00,4


December 26th (Wednesday), 16:30 - 17:32, 4 guests.

1. Start:
    - 4 Dishes:
        - Salmon x 2
        - Sirolin Steak
        - Turkey Bologn
    - 1 Beer:
        - Goose Island
    - 3 Wines:
        - Black River
        - Chateau Char x 2

`After Work`

#### 9. Table 360402335

In [80]:
silvester[silvester.index == 360402335].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
360402335,2018-12-27 18:18:52,2018-12-27 19:49:49,01:30:57,2018-12-27 18:19:13,5ozDriftng CHARD,6.0,0,0,0.000,1.0,7.50,8.475289,3
360402335,2018-12-27 18:18:52,2018-12-27 19:49:49,01:30:57,2018-12-27 18:19:13,8oz Pinot Grigio,6.0,0,0,0.000,1.0,10.00,11.300386,3
360402335,2018-12-27 18:18:52,2018-12-27 19:49:49,01:30:57,2018-12-27 18:19:13,16oz Shock Top,1.0,267,2,0.473,1.0,6.75,7.627760,3
360402335,2018-12-27 18:18:52,2018-12-27 19:49:49,01:30:57,2018-12-27 18:28:53,Salmon Teriyaki,2.0,0,0,0.000,1.0,18.99,21.459432,3
360402335,2018-12-27 18:18:52,2018-12-27 19:49:49,01:30:57,2018-12-27 18:28:53,Salmon Teriyaki,2.0,0,0,0.000,1.0,18.99,21.459432,3
360402335,2018-12-27 18:18:52,2018-12-27 19:49:49,01:30:57,2018-12-27 18:28:53,PB House Burger,2.0,0,0,0.000,1.0,14.99,16.939278,3
360402335,2018-12-27 18:18:52,2018-12-27 19:49:49,01:30:57,2018-12-27 19:21:55,Froyo Strawberry,2.0,0,0,0.000,1.0,2.99,3.378815,3
360402335,2018-12-27 18:18:52,2018-12-27 19:49:49,01:30:57,2018-12-27 19:46:43,Coffee Decaf,4.0,0,0,0.000,1.0,2.69,3.039804,3
360402335,2018-12-27 18:18:52,2018-12-27 19:49:49,01:30:57,2018-12-27 19:46:43,Coffee,4.0,0,0,0.000,1.0,2.69,3.039804,3


December 27th (Thursday), 18:18 - 19:49, 3 guests.

1. Start:
    - 2 Wines:
        - Drifting Chardonnay
        - Pinot grigio
    - 1 Beer:
        - Shock Top
2. 10 Minutes Later:
    - 3 Dishes:
        - Salmon x 2
        - Burger
3. 50 Minutes Later:
    - 1 Dish:
        - Froyo Strawberry
4. 25 Minutes Later:
    - 2 Soft Drinks:
        - Coffee x 2

`Dinner`

#### 10. Table 361369624

In [81]:
silvester[silvester.index == 361369624].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
361369624,2018-12-29 19:31:00,2018-12-29 20:30:15,00:59:15,2018-12-29 19:31:00,WATER.,4.0,0,0,0.0,1.0,0.00,0.000000,5
361369624,2018-12-29 19:31:00,2018-12-29 20:30:15,00:59:15,2018-12-29 19:31:00,WATER.,4.0,0,0,0.0,1.0,0.00,0.000000,5
361369624,2018-12-29 19:31:00,2018-12-29 20:30:15,00:59:15,2018-12-29 19:31:00,WATER.,4.0,0,0,0.0,1.0,0.00,0.000000,5
361369624,2018-12-29 19:31:00,2018-12-29 20:30:15,00:59:15,2018-12-29 19:31:00,WATER.,4.0,0,0,0.0,1.0,0.00,0.000000,5
361369624,2018-12-29 19:31:00,2018-12-29 20:30:15,00:59:15,2018-12-29 19:31:00,WATER.,4.0,0,0,0.0,1.0,0.00,0.000000,5
361369624,2018-12-29 19:31:00,2018-12-29 20:30:15,00:59:15,2018-12-29 19:42:00,ASIAGO.BOWTIE,2.0,0,0,0.0,1.0,19.99,22.590156,5
361369624,2018-12-29 19:31:00,2018-12-29 20:30:15,00:59:15,2018-12-29 19:42:00,PAD THAI,2.0,0,0,0.0,1.0,0.00,0.000000,5
361369624,2018-12-29 19:31:00,2018-12-29 20:30:15,00:59:15,2018-12-29 19:42:00,ADD PAN BREAD,2.0,0,0,0.0,1.0,5.25,5.932882,5
361369624,2018-12-29 19:31:00,2018-12-29 20:30:15,00:59:15,2018-12-29 19:42:00,DELUXE.BURGER,2.0,0,0,0.0,1.0,17.73,20.036191,5


December 29th (Saturday), 19:31 - 20:30, 5 guests.

1. Start:
    - 5 Water
2. 10 Minutes Later:
    - 5 Dishes:
        - Asiago BowTie
        - Burger x 2
        - Pad Thai Chicken
        - Fajita Steak
    - 1 Beer:
        - Brewsky Beer

`Dinner`

-----

### Conclusion

The relevant occasions are:
1. ~~`Business Date`~~
2. ~~`Dinner Date`~~
3. `Social Gathering`
4. `Family Event`
5. ~~`Party`~~
6. `Drinking`
7. `After Work`
8. `Lunch`
9. `Dinner`

Or:
1. `Social Gathering`  
    No time-specific event.  
    Contains beers / shareable dishes / many meal steps.
2. `Family Event`  
    Contains **K.** / **K-** or any other kids-related dish (pancakes, milkshake etc.).  
3. `Drinking`  
    A lot of alcoholic drinks compared to number of dishes.
4. `After Work`  
    After work hours, less shareable dishes compared to a social-gathering event, contains more main dishes.
5. `Lunch`  
    Lunch period, mostly contains main dishes rather than desserts / alcohol.  
    Number of main dishes usually equals to the number of guests.
6. `Dinner`  
    Dinner period, mostly contains main dishes rather than desserts / alcohol.  
    Number of main dishes usually equals to the number of guests.